# 챗봇 만들기

### https://jfun.tistory.com/199?category=851249
### https://www.slideshare.net/KimSungdong1/20170227-72644192

In [2]:
input_data = '판교에 지금 주문해줘'
output_data = ''

request = {
    "intent_id":"",
    "input_data":input_data,
    "request_type":"text",
    "story_slot_entity":{},
    "output_data":output_data
}

## 기본 데이터 셋(DB))

In [3]:
intent_list = {
    "주문":["주문", "배달"],
    "예약":["예약", "잡아줘"],
    "정보":["정보","알려"]
}

story_slot_entity = {"주문":{"메뉴":None, "장소":None, "날짜":None},
"예약":{"장소": None, "날짜":None},
"정보":{"대상":None}
}

## 형태소 분석

In [4]:
from eunjeon import Mecab
mecab = Mecab()
preprocessed = mecab.pos(request.get('input_data'))

print(preprocessed)

[('판교', 'NNG'), ('에', 'JKB'), ('지금', 'MAG'), ('주문', 'NNG'), ('해', 'XSV+EC'), ('줘', 'VX+EC')]


## Intent 도출(Ruled Based)

In [5]:
# Char CNN을 사용해서 연결하면 좀 더 쉽게 만들 수 있다
intent_id = "주문"
slot_value = story_slot_entity.get("주문")

## NER 도출(Ruled based)

In [6]:
# LSTM 기법을 사용해서 연결하면 좀 더 쉬울 수 있다
menu_list = ["피자", "햄버거", "치킨"]
loc_list = ["판교", "야탑", "서현"]
date_list = ["지금", '내일', '모레']

## Dictionary 기반 slot 구성

In [7]:
for pos_tag in preprocessed:
    if pos_tag[1] in ['NNG', 'NNP', 'SL', 'MAG']:
        if pos_tag[0] in menu_list:
            slot_value['메뉴'] = pos_tag[0]
        elif pos_tag[0] in loc_list:
            slot_value['장소'] = pos_tag[0]
        elif pos_tag[0] in date_list:
            slot_value['날짜'] = pos_tag[0]


print(story_slot_entity.get('주문'))

{'메뉴': None, '장소': '판교', '날짜': '지금'}


## 빈 slot 검색

In [8]:
if None in slot_value.values():
    key_values=""
    for key in slot_value.keys():
        if slot_value[key] is None:
            key_values = key_values + key + ""
    output_data = key_values + "선택해주세요"
else:
    output_data="주문이 완료 되었습니다."

print(output_data)

메뉴선택해주세요


In [9]:
response = {
    "intent_id":"",
    "input_data":input_data,
    "request_type":"text",
    "story_slot_entity":{},
    "output_data":""
}

response["output_data"]=output_data

print(response["output_data"])

메뉴선택해주세요


# Word Representation

In [48]:
ona_data = [
    ['안녕', '만나서 반가워'],
    ['넌 누구니','나는 AI봇이란다.'],
    ['피자 주문할께', '음료도 주문해줘'],
    ['음료는 뭘로', '콜라로 해줘']
]

train_gata = list(map(lambda x: mecab.morphs(''.join(x)), ona_data))

import itertools

train_data = list(itertools.chain.from_iterable(train_gata))

print(list(train_data))

['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '봇', '이', '란다', '.', '피자', '주문', '할께', '음료', '도', '주문', '해', '줘', '음료', '는', '뭘로', '콜라', '로', '해', '줘']


In [49]:
import numpy as np

bucket = np.zeros(len(train_data), dtype=np.float)
print(bucket)

for word in train_data:
    bucket_temp = bucket.copy()
    bucket_temp = np.insert(bucket_temp, train_data.index(word), 1)
    print(bucket_temp)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


# Word2Vec

In [50]:
from gensim.models import word2vec

train_data = [train_data]
print(train_data)

model = word2vec.Word2Vec(size=50, window=2, min_count=1)
model.build_vocab(train_data)
model.train(train_data, epochs=model.iter,total_examples=model.corpus_count)
print("model check:{0}".format(model))

[['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '봇', '이', '란다', '.', '피자', '주문', '할께', '음료', '도', '주문', '해', '줘', '음료', '는', '뭘로', '콜라', '로', '해', '줘']]
model check:Word2Vec(vocab=24, size=50, alpha=0.025)


In [51]:
import os
model.save(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\w2v.bin")
model = word2vec.Word2Vec.load(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\w2v.bin")
print("model load check : {0}".format(model))

model load check : Word2Vec(vocab=24, size=50, alpha=0.025)


In [62]:
X = model[]
print(model.wv.index2word)

Word2Vec(vocab=24, size=50, alpha=0.025)
['는', '주문', '음료', '해', '줘', '안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', 'AI', '봇', '이', '란다', '.', '피자', '할께', '도', '뭘로', '콜라', '로']


In [55]:
print(model["안녕"])

[-0.00976191  0.00973107  0.00252566  0.00820518 -0.00646039  0.00390535
 -0.00628025 -0.00704649 -0.00706708 -0.00498548  0.00822745  0.00053697
  0.00817156 -0.00821855 -0.00402307 -0.00452205 -0.00876464 -0.00399296
 -0.00167992 -0.00735352  0.00124907 -0.00224733  0.00192989  0.00914724
 -0.00956092  0.00481127  0.00788746 -0.00258857 -0.00404193 -0.00562656
 -0.00486423 -0.0033742  -0.00810599  0.0035026   0.00563072  0.00184724
  0.00378866  0.00454431  0.00513543 -0.00544446  0.00536345  0.00793488
  0.00616158  0.00824817 -0.00291877  0.00648411  0.00172757 -0.00349816
 -0.00697151  0.00120333]


In [57]:
print(model["AI"])

[ 0.00988131  0.00828089  0.00588054 -0.00300516  0.00822219 -0.00365124
 -0.00801502  0.00072754 -0.00099351  0.00973184  0.0097055  -0.00023882
  0.0067852  -0.00537235  0.00191611  0.00774843 -0.00862572  0.00496778
 -0.00430471  0.00445049 -0.00469619  0.00640069 -0.00866665 -0.00337163
  0.00824308  0.00715727  0.00148865 -0.00540752  0.00841281  0.00533213
  0.00207207 -0.00114708  0.00017083  0.00141542 -0.00453451  0.00323375
 -0.00685723 -0.00848806 -0.00481923  0.00331312 -0.00862415  0.00179881
 -0.00304985  0.00922066 -0.00756584  0.00641558  0.00413202 -0.00847561
  0.00806909 -0.00329407]


In [58]:
res1 = model.most_similar(positive="누구", negative="", topn=10)
print(res1)

[('주문', 0.29700160026550293), ('니', 0.26338884234428406), ('만나', 0.2469508945941925), ('콜라', 0.18749526143074036), ('봇', 0.1744612157344818), ('할께', 0.1415926218032837), ('나', 0.13562972843647003), ('도', 0.0877845361828804), ('이', 0.05164101719856262), ('AI', 0.02810666523873806)]


In [59]:
from sklearn.manifold import TSNE
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

font_name = matplotlib.font_manager.FontProperties(
    fname=r"C:\Program Files\Android\Android Studio\plugins\android\lib\layoutlib\data\fonts\NanumGothic.ttf"
).get_name()
vocab = model.wv.index2word
matplotlib.rc('font', family=font_name)
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)
df = pd.concat([pd.DataFrame(X_tsne), pd.Series(vocab)], axis=1)

df.columns = ['x', 'y', 'word']
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
print(df)
ax.scatter(df['x'], df['y'])
ax.set_xlim(df['x'].max(), df['x'].min())
ax.set_ylim(df['y'].max(), df['y'].min())
for i, txt in enumerate(df['word']):
    ax.annotate(txt, (df['x'].iloc[i], df['y'].iloc[i]))
plt.show()

NameError: name 'X' is not defined